In [5]:
import os, random
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.all import *

In [6]:
# specify input video file path
input_file = 'D:\MOVIES\Thor Love And Thunde 2022 [Hindi-English].mkv'

# specify output folder to save split videos
output_folder = './output/'

# specify duration of each split video (in seconds)
split_duration = 420

# specify noise intensity
noise_intensity = 0.05

In [ ]:
# create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# create VideoFileClip object from input video
clip = VideoFileClip(input_file)

# calculate total duration of input video (in seconds)
total_duration = clip.duration

# calculate number of split videos required
num_splits = int(total_duration / split_duration) + 1

# loop through each split and save as separate mp4 file
for i in range(num_splits):
    # calculate start and end time for this split
    start_time = i * split_duration
    end_time = min((i + 1) * split_duration, total_duration)
    
    # extract video clip for this split
    subclip = clip.subclip(start_time, end_time)
    
    # add horizontal flip effect to some frames
    flipped = [subclip.fl_image(lambda frame: frame[:, ::-1]) if random.random() < 0.2 else subclip]
    
    # add random noise effect to all frames
    noisy = [vfx.lum_contrast(gaussian_noise(f, noise_intensity), lum=1.2, contrast=1.2) for f in flipped]

    # specify output file name for this split
    output_file = os.path.join(output_folder, f'split_{i}.mp4')
    
    # concatenate modified video clips and write to output file
    concat_clip = concatenate_videoclips(noisy)
    concat_clip.write_videofile(output_file, codec='libx264')
    
    # close subclip and concat_clip objects to free up memory
    subclip.close()
    concat_clip.close()
    break

# close input clip object to free up memory
clip.close()

# print message when all splits are saved
print(f'{num_splits} split videos saved in {output_folder}')


In [8]:
import os
import random
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.all import *

# create VideoFileClip object from input video
clip = VideoFileClip(input_file)

# calculate total duration of input video (in seconds)
total_duration = clip.duration

# calculate number of split videos required
num_splits = int(total_duration / split_duration) + 1

# loop through each split and save as separate mp4 file
for i in range(num_splits):
    # calculate start and end time for this split
    start_time = i * split_duration
    end_time = min((i + 1) * split_duration, total_duration)
    
    # extract video clip for this split
    subclip = clip.subclip(start_time, end_time)
    
    # add horizontal flip effect to some frames
    flipped = [subclip.fl_image(lambda frame: frame[:, ::-1]) if random.random() < 0.2 else subclip]
    
    # add random noise effect to all frames
    noisy = [lum_contrast(gaussian_noise(f, noise_intensity), lum=1.2, contrast=1.2) for f in flipped]

    # specify output file name for this split
    output_file = os.path.join(output_folder, f'split_{i}.mp4')
    
    # concatenate modified video clips and write to output file
    concat_clip = concatenate_videoclips(noisy)
    concat_clip.write_videofile(output_file, codec='libx264')
    
    # close subclip and concat_clip objects to free up memory
    subclip.close()
    concat_clip.close()

# close input clip object to free up memory
clip.close()

# print message when all splits are saved
print(f'{num_splits} split videos saved in {output_folder}')


NameError: name 'gaussian_noise' is not defined